In [1]:
import polars as pl
import pandas as pd
from psifr import fr
data_file = "2013_Morton.csv"

In [2]:
raw = pd.read_csv(data_file)
data = fr.merge_free_recall(raw, list_keys=["session", "list_type", "list_category"], study_keys=["category", "response", "response_time"])
data

,subject,list,item,input,output,study,recall,repeat,intrusion,session,list_type,list_category,category,response,response_time,prior_list,prior_input
0,1,1,TOWEL,1.0,13.0,True,True,0,False,1,pure,obj,obj,3.0,1.517,NaN,NaN
1,1,1,LADLE,2.0,NaN,True,False,0,False,1,pure,obj,obj,3.0,1.404,NaN,NaN
2,1,1,THERMOS,3.0,NaN,True,False,0,False,1,pure,obj,obj,3.0,0.911,NaN,NaN
3,1,1,LEGO,4.0,18.0,True,True,0,False,1,pure,obj,obj,3.0,0.883,NaN,NaN
4,1,1,BACKPACK,5.0,10.0,True,True,0,False,1,pure,obj,obj,3.0,0.819,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47861,47,48,ALEX RODRIGUEZ,20.0,6.0,True,True,0,False,3,mixed,mixed,cel,3.0,1.282,NaN,NaN
47862,47,48,AL PACINO,21.0,4.0,True,True,0,False,3,mixed,mixed,cel,3.0,0.867,NaN,NaN
47863,47,48,NELSON MANDELA,22.0,3.0,True,True,0,False,3,mixed,mixed,cel,2.0,1.261,NaN,NaN
47864,47,48,VIN DIESEL,23.0,2.0,True,True,0,False,3,mixed,mixed,cel,3.0,1.213,NaN,NaN


In [3]:
df = (
    pl.DataFrame(data)
    .with_columns(
        pl.col(["input", "output", "response", "prior_list", "prior_input"]).cast(pl.Int64),
    )
    .drop("prior_list", "prior_input")
    .select("subject", "session", "list", "item", "input", "output", "study", "recall", "repeat", "intrusion", "list_type", "list_category", "response", "response_time")
)
df.head()

subject,session,list,item,input,output,study,recall,repeat,intrusion,list_type,list_category,response,response_time
i64,i64,i64,str,i64,i64,bool,bool,i64,bool,str,str,i64,f64
1,1,1,"""TOWEL""",1,13,true,true,0,false,"""pure""","""obj""",3,1.517
1,1,1,"""LADLE""",2,null,true,false,0,false,"""pure""","""obj""",3,1.404
1,1,1,"""THERMOS""",3,null,true,false,0,false,"""pure""","""obj""",3,0.911
1,1,1,"""LEGO""",4,18,true,true,0,false,"""pure""","""obj""",3,0.883
1,1,1,"""BACKPACK""",5,10,true,true,0,false,"""pure""","""obj""",3,0.819


In [4]:
df.write_csv("2013_Morton_scored.csv", null_value="n/a")